### Imports


In [6]:
import numpy as np
import pandas as pd
import SimpleITK as sitk
import os
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import matplotlib.image as mpimg
from scipy import ndimage
from scipy.stats import gaussian_kde
import seaborn as sns
from skimage import morphology



%matplotlib widget

### Reading Data

In [ ]:
def load_itk_image(filename):
    itkimage = sitk.ReadImage(filename)

    numpyImage = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))

    return numpyImage, numpyOrigin, numpySpacing
def read_BMP_3D_dataset(root_dir):
    Dataset={}
    subsets = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

    for subset in subsets:
        bmp_files = [f for f in os.listdir(os.path.join(root_dir, subset)) if f.endswith(".bmp")]
        if subset not in Dataset:
            Dataset[subset] = {}

        for index, file in enumerate(bmp_files):
            Dataset[subset][f"CT_{index}"] = file

    return Dataset

In [14]:
if __name__ == "__main__":
    # Charger le dataset
    Data = read_BMP_3D_dataset("Data/BMP_3D")
    
    # Exemple : Accéder à une valeur spécifique
    subset_name = "0001"  # Nom du subset que tu veux explorer
    file_key = "CT_0"     # Clé du fichier dans ce subset

    # Accéder à la valeur
    if subset_name in Data:
        subset_value = Data[subset_name]  # Accède aux fichiers dans le subset
        if file_key in subset_value:
            value_ = subset_value[file_key]  # Cela correspond au nom du fichier BMP
            print(f"Le fichier {file_key} dans le subset {subset_name} est : {value_}")
        else:
            print(f"Clé {file_key} non trouvée dans le subset {subset_name}.")
    else:
        print(f"Subset {subset_name} non trouvé.")


Le fichier CT_0 dans le subset 0001 est : 0.bmp


In [18]:
def load_csv(filename):
    return pd.read_csv(f"evaluationScript/annotations/{filename}")

In [19]:
def basic_statistics(data):
    print("Infos:\n", data.info())
    print("Description:\n", data.describe())
    print("Presence of null values:\n")
    print(data.isnull().sum())

In [20]:
if __name__ == "__main__":
    annotations = load_csv("anno_WorldCoord.csv")
    basic_statistics(annotations)
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           330 non-null    int64  
 1   image        330 non-null    object 
 2   x_center     330 non-null    float64
 3   y_center     330 non-null    float64
 4   z_center     330 non-null    float64
 5   radius_mm    330 non-null    float64
 6   diameter_mm  330 non-null    object 
 7   label        330 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 20.8+ KB
Infos:
 None
Description:
                id    x_center    y_center     z_center   radius_mm
count  330.000000  330.000000  330.000000   330.000000  330.000000
mean    55.375758   -3.969165  -51.596845  -288.401122    9.036256
std     24.939770   73.593000   89.756687   830.611775    5.320981
min      1.000000 -117.597000 -217.197000 -1157.300000    1.607200
25%     38.250000  -68.404525 -129.727975  -854.675